# Setup

In [5]:
pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\maks_\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [6]:
import json
import pandas as pd
import spacy
from spacy import displacy
import locale

# Data preparation

In [7]:
train_url = 'https://raw.githubusercontent.com/sonodug/NER/main/data/train.json'
test_url = 'https://raw.githubusercontent.com/sonodug/NER/main/data/test.json'

train_df  = pd.read_json(train_url)
test_df = pd.read_json(test_url)

In [8]:
train_df.head()

,id,text,label,extracted_part
0,809436509,Извещение о проведении открытого конкурса в эл...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
1,854885310,ТРЕБОВАНИЯ К СОДЕРЖАНИЮ ЗАЯВКИ участника запро...,обеспечение исполнения контракта,{'text': ['Поставщик должен предоставить обесп...
2,4382157,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
3,184555082,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...
4,211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,{'text': ['Размер обеспечения исполнения контр...


In [9]:
test_df.head()

,id,text,label
0,762883279,МУНИЦИПАЛЬНЫЙ КОНТРАКТ № ______ на оказание ус...,обеспечение исполнения контракта
1,311837655,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта
2,540954893,Идентификационный код закупки: 222633005300163...,обеспечение исполнения контракта
3,274660397,Идентификационный код закупки: 222631202689463...,обеспечение исполнения контракта
4,732742591,Идентификационный код закупки: 222637800031163...,обеспечение исполнения контракта


In [15]:
len(train_df["text"].values)

1799

In [16]:
len(test_df["text"].values)

318

In [10]:
locale.getpreferredencoding = lambda: "UTF-8"

# NER with spaCy's pretrained model 

In [11]:
!python -m spacy download ru_core_news_sm

     ---------------------------------------- 15.3/15.3 MB 4.6 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


You should consider upgrading via the 'C:\Users\maks_\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [19]:
nlp = spacy.load('ru_core_news_sm')
ner = nlp.get_pipe("ner")

nlp.pipe_names

['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [18]:
ner.labels

('LOC', 'ORG', 'PER')

In [13]:
ex_text = """В 1999 году я побывал в Париже и посетил Эйфелеву башню. Там было много туристов, и я сделал много фотографий.
Вечером мы с моим другом Владимиром пошли в ресторан и заказали французский сыр и красное вино. 
Потом мы прогулялись по набережной Сены и наслаждались красивым видом на город.
Вообще, Париж очень интересный город, и я был рад, что смог побывать там."""
doc = nlp(ex_text)

# The NER pipeline component tags entities in the doc with various attributes
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Париже 24 30 LOC
Эйфелеву башню 41 55 LOC
Владимиром 136 146 PER
Сены 243 247 LOC
Париж 296 301 LOC


In [14]:
# Use displacy to visualize entities
displacy.render(doc, style='ent', jupyter=True)

In [20]:
formatted_data = []

for index, row in train_df.iterrows():
    text = row['text']
    entities = []
    for i in range(len(row['extracted_part']['text'])):
      start = row['extracted_part']['answer_start'][0]
      end = row['extracted_part']['answer_end'][0]
      label = row['label']
      entities.append((start, end, label))
    formatted_data.append((text, {"entities": entities}))

print(formatted_data[0])

('Извещение о проведении открытого конкурса в электронной форме для закупки №0328300032822000806 Общая информация Номер извещения 0328300032822000806 Наименование объекта закупки Поставка продуктов питания Способ определения поставщика (подрядчика, исполнителя) Открытый конкурс в бль Порядок внесения денежных средств в качестве обеспечения заявки на участие в закупке, а также условия гарантии Обеспечение заявки на участие в закупке может предоставляться участником закупки в виде денежных средств или независимой гарантии, предусмотренной ст. 45 Федерального закона № 44-ФЗ. Выбор способа обеспечения осуществляется участником закупки самостоятельно. Срок действия независимой гарантии должен составлять не менее месяца с даты окончания срока подачи заявок. Обеспечение заявки на участие в закупке предоставляется в соответствии с ч. 5 ст. 44 Федерального закона № 44-ФЗ. Условия независимой гарантии в соответствии со ст. 45 Федерального закона № 44-ФЗ. Реквизиты счета в соответствии с п.16 ч. 

In [21]:
for _, annotations in formatted_data:
  for ent in annotations.get("entities"):
    ner.add_label(ent[2])

In [22]:
ner.labels

('LOC',
 'ORG',
 'PER',
 'обеспечение гарантийных обязательств',
 'обеспечение исполнения контракта')

In [24]:
move_names = list(ner.move_names)
move_names

['B-ORG',
 'B-PER',
 'B-LOC',
 'I-ORG',
 'I-PER',
 'I-LOC',
 'L-ORG',
 'L-PER',
 'L-LOC',
 'U-ORG',
 'U-PER',
 'U-LOC',
 'O',
 'B-обеспечение исполнения контракта',
 'I-обеспечение исполнения контракта',
 'L-обеспечение исполнения контракта',
 'U-обеспечение исполнения контракта',
 'B-обеспечение гарантийных обязательств',
 'I-обеспечение гарантийных обязательств',
 'L-обеспечение гарантийных обязательств',
 'U-обеспечение гарантийных обязательств']

# Train a custom model

In [23]:
iterations = 24

loss_scores = []
accuracy_scores = []
precision_scores = []
recall_scores = []
f1_scores = []
data = []

In [ ]:
%%time

import random
from spacy.util import minibatch, compounding
from pathlib import Path
from spacy.training.example import Example
from tqdm import tqdm
from spacy.scorer import Scorer
import warnings

pipe_exceptions = ["ner", "trf_wordpiecer", "trf_tok2vec"]
unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe not in pipe_exceptions]

# TRAINING THE MODEL
with nlp.disable_pipes(*unaffected_pipes):
  warnings.filterwarnings("ignore", category=UserWarning, module='spacy') 
  scorer = Scorer()
  # Training for 30 iterations
  for iteration in range(1, iterations):
    # shuffling examples before every iteration
    examples = []
    random.shuffle(formatted_data)
    losses = {}
    # batch up the examples using spaCy's minibatch
    batches = minibatch(formatted_data, size=compounding(4.0, 32.0, 1.001))
    progress_bar = tqdm(total=len(formatted_data), desc=f"Iteration {iteration}", leave=False)
    for batch in batches:
      for text, annotations in batch:
        doc = nlp.make_doc(text)
        predict = nlp(text)
        ex = Example.from_dict(predict, annotations)
        example = Example.from_dict(doc, annotations)
        ex.predicted = nlp(str(ex.predicted))
        examples.append(ex)
        nlp.update([example], losses=losses, drop=0.3)
        tqdm._instances.clear()
        progress_bar.update(1)

    metrics = scorer.score_spans(examples, "ents")
    print(metrics)
    loss_scores.append(losses['ner'])
    precision_scores.append(metrics['ents_p'])
    recall_scores.append(metrics['ents_r'])
    f1_scores.append(metrics['ents_f'])
    data.append([iteration, losses['ner'], metrics['ents_p'], metrics['ents_r'], metrics['ents_f']])
    print('  Losses:', losses['ner'])
    print()
    examples = []